In [11]:
import string
import nltk
import senticnet

nltk.download('stopwords')

sentence = "Could you please not take this? John does not likes to watch movies. Mary does not likes movies from Austria. :) (see under http://rogers.li) #lovemovie N"
sentence2 = "John like watch movies . Mary likes movies . happy"

pos_map = {} # not used!
pos_map["NN"] = "n"
pos_map["NNP"] = "n"
pos_map["NNS"] = "n"
pos_map["NNPS"] = "n"
pos_map["NNP"] = "n"
pos_map["VB"] = "v"
pos_map["VBD"] = "v"
pos_map["VBG"] = "v"
pos_map["VBN"] = "v"
pos_map["VBP"] = "v"
pos_map["VBZ"] = "v"
pos_map["JJ"] = "a"
pos_map["JJR"] = "a"
pos_map["JJS"] = "a"
pos_map["RB"] = "r"
pos_map["RBR"] = "r"
pos_map["RBS"] = "r"

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\COMMANDER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def nltk_tag_to_wordnet_tag(nltk_tag):
    from nltk.corpus import wordnet
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None


In [3]:
def remove_stopwords(word_tokens):
    from nltk.corpus import stopwords

    stop_words = set(stopwords.words('english'))
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    return filtered_sentence

In [4]:
def build_bow(sentence: str):
    """
    Build a Bag of words from a sentence
    """
    from keras.preprocessing.text import Tokenizer
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(sentence)
    sequences = tokenizer.texts_to_sequences(sentence)
    word_index = tokenizer.word_index
    bow = {}
    for key in word_index:
        bow[key] = sequences[0].count(word_index[key])
    return bow


In [5]:
def print_bow(bow: {}):
    """
    Print a bag of words
    :param bow: A bag od words to print
    :return:
    """
    print(f"Bag of word sentence 1 :\n{bow}")

In [6]:
def tokenize(sentence: str):
    from nltk.tokenize import word_tokenize
    result_tokens = word_tokenize(sentence)
    return result_tokens

In [7]:
def lemm_tokens(word_tokens):
    from nltk import pos_tag
    from nltk.stem import WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()

    pos_tokens = pos_tag(word_tokens)
    print(pos_tokens)
    result_tokens = []
    # Map to
    # first element is the word, second element is the pos tag
    for token in pos_tokens :

        # skip not mapped speech
        if token[1].upper() != token[0].upper():
            # Get Part of Speech from pos_tag to relevant tag of the lemmatizer
            # example 'NNP' => 'n'
            pos = nltk_tag_to_wordnet_tag(token[1])
        else:
            pos = ""
        if pos != "" and pos is not None:
            result_tokens.append(lemmatizer.lemmatize(token[0], pos=pos))
        else:
             result_tokens.append(token[0])
    return result_tokens

In [8]:
def get_emotions(tokens):
    from senticnet.senticnet import SenticNet
    result = {}
    sn = SenticNet()
    for token in tokens:
        moodtags = ""

        if token in sn.data:
            moodtags = sn.moodtags(token)
            print(token, moodtags)
    #TODO
    return result

In [9]:
# Script
print(sentence)
# get Bow
# remove stopwords -> remove special string/words -> Get POS -> Lemm -> lookup emotion for each word -> w emotions
tokens1 = tokenize(sentence)
print(tokens1)

tokens2 = lemm_tokens(tokens1)
print(tokens2)

tokens3 = remove_stopwords(tokens2)
#sentence2 = sentence
print(' '.join(tokens3))
print_bow(build_bow([sentence]))
print("=======================================")
get_emotions(tokens3)

Could you please not take this? John does not likes to watch movies. Mary does not likes movies from Austria. :) (see under http://rogers.li) #lovemovie N
['Could', 'you', 'please', 'not', 'take', 'this', '?', 'John', 'does', 'not', 'likes', 'to', 'watch', 'movies', '.', 'Mary', 'does', 'not', 'likes', 'movies', 'from', 'Austria', '.', ':', ')', '(', 'see', 'under', 'http', ':', '//rogers.li', ')', '#', 'lovemovie', 'N']
[('Could', 'NNP'), ('you', 'PRP'), ('please', 'VBP'), ('not', 'RB'), ('take', 'VB'), ('this', 'DT'), ('?', '.'), ('John', 'NNP'), ('does', 'VBZ'), ('not', 'RB'), ('likes', 'VB'), ('to', 'TO'), ('watch', 'VB'), ('movies', 'NNS'), ('.', '.'), ('Mary', 'NNP'), ('does', 'VBZ'), ('not', 'RB'), ('likes', 'VB'), ('movies', 'NNS'), ('from', 'IN'), ('Austria', 'NNP'), ('.', '.'), (':', ':'), (')', ')'), ('(', '('), ('see', 'VB'), ('under', 'IN'), ('http', 'NN'), (':', ':'), ('//rogers.li', 'NN'), (')', ')'), ('#', '#'), ('lovemovie', 'JJ'), ('N', 'NNP')]
['Could', 'you', 'pleas

Using TensorFlow backend.


Bag of word sentence 1 :
{'not': 3, 'does': 2, 'likes': 2, 'movies': 2, 'could': 1, 'you': 1, 'please': 1, 'take': 1, 'this': 1, 'john': 1, 'to': 1, 'watch': 1, 'mary': 1, 'from': 1, 'austria': 1, 'see': 1, 'under': 1, 'http': 1, 'rogers': 1, 'li': 1, 'lovemovie': 1, 'n': 1}
please ['#interest', '#admiration']
take ['#interest', '#admiration']
watch ['#joy', '#interest']
movie ['#joy', '#joy']
movie ['#joy', '#joy']
see ['#interest', '#admiration']


{}

In [13]:
print(' '.join(tokens3))

Could please take ? John like watch movie . Mary like movie Austria . : ) ( see http : //rogers.li ) # lovemovie N


In [ ]:
# ab hier kommt senticnet zum einsatz